In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
df = pd.read_csv("NEW_merged_company_financial.csv")
df.head()

,Unnamed: 0,Tax_number,NUTS2,sector,HQ_Site_Branch,Report Start Date,Report End Date,Total Net Sales,Cost of Goods Sold,Gross Income,...,Profit After Tax IS,Profit or Loss of The Year IS,Total Assets,Shareholder's Equity,Long-Term Liabilities,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses
0,0,10568187.0,Pest,machinery,HQ,2003-01-01,2003-12-31,46109000.0,20296000.0,1531000.0,...,1558000.0,1558000.0,4752000.0,6971000.0,0.0,3028000.0,10365000.0,0.327862,14.373938,0.446056
1,1,10568187.0,Pest,machinery,HQ,2004-01-01,2004-12-31,40669000.0,17226000.0,6428000.0,...,6462000.0,6462000.0,9947000.0,7145000.0,0.0,2508000.0,9966000.0,0.649643,6.454103,0.367818
2,2,10695889.0,Southern Transdanubia,machinery,HQ,2003-01-01,2003-12-31,10530000.0,3330000.0,-949000.0,...,-948000.0,-948000.0,13236000.0,9258000.0,0.0,3955000.0,13329000.0,-0.071623,inf,0.275184
3,3,10695889.0,Southern Transdanubia,machinery,HQ,2004-01-01,2004-12-31,13954000.0,2638000.0,2510000.0,...,1946000.0,1946000.0,14776000.0,10809000.0,0.0,3942000.0,14811000.0,0.131700,inf,0.214123
4,4,10695889.0,Southern Transdanubia,machinery,HQ,2005-01-01,2005-09-01,1027000.0,760000.0,-455000.0,...,4666000.0,4666000.0,8200000.0,7177000.0,0.0,1046000.0,8223000.0,0.569024,inf,0.478891


In [3]:
# Drop columns
df = df.drop(columns=["Unnamed: 0", 'Report End Date', 'HQ_Site_Branch'])

# convert tax number column type
df['Tax_number'] = pd.Categorical(df.Tax_number)

# convert date column type
df['Report Start Date'] = pd.to_datetime(df['Report Start Date'])

# unitfy time unit to year
df['Report Start Date'] = df['Report Start Date'].dt.strftime('%Y')
df.head()

,Tax_number,NUTS2,sector,Report Start Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,Regular Operations Income,Extraordinary Income,Profit After Tax IS,Profit or Loss of The Year IS,Total Assets,Shareholder's Equity,Long-Term Liabilities,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses
0,10568187.0,Pest,machinery,2003,46109000.0,20296000.0,1531000.0,6000.0,1537000.0,-40000.0,1558000.0,1558000.0,4752000.0,6971000.0,0.0,3028000.0,10365000.0,0.327862,14.373938,0.446056
1,10568187.0,Pest,machinery,2004,40669000.0,17226000.0,6428000.0,34000.0,6462000.0,0.0,6462000.0,6462000.0,9947000.0,7145000.0,0.0,2508000.0,9966000.0,0.649643,6.454103,0.367818
2,10695889.0,Southern Transdanubia,machinery,2003,10530000.0,3330000.0,-949000.0,1000.0,-948000.0,0.0,-948000.0,-948000.0,13236000.0,9258000.0,0.0,3955000.0,13329000.0,-0.071623,inf,0.275184
3,10695889.0,Southern Transdanubia,machinery,2004,13954000.0,2638000.0,2510000.0,-480000.0,2030000.0,0.0,1946000.0,1946000.0,14776000.0,10809000.0,0.0,3942000.0,14811000.0,0.131700,inf,0.214123
4,10695889.0,Southern Transdanubia,machinery,2005,1027000.0,760000.0,-455000.0,121000.0,-334000.0,-5000000.0,4666000.0,4666000.0,8200000.0,7177000.0,0.0,1046000.0,8223000.0,0.569024,inf,0.478891


In [4]:
# unify monthly/quaterly profit values into annual ones
annual_profit = pd.DataFrame(df.groupby(['Tax_number', 'Report Start Date'])['Profit After Tax IS'].sum())

# rename column to annual_profit
annual_profit.rename(columns = {'Profit After Tax IS': 'annual_profit'}, inplace=True)

# check the result
annual_profit.head()
# Note: groupby() adds non-existing years with 0.0

annual_profit
Tax_number Report Start Date               
10568187.0 2001                         0.0
           2002                         0.0
           2003                   1558000.0
           2004                   6462000.0
           2005                         0.0

In [5]:
# Normalize annual_profit
# 1. replace 0.0 with nans. Otherwise, avergaes will be dragged down. 
annual_profit['annual_profit'].replace(0.0, np.NaN)

# 2. calculate avg annual_profit 
profit_mean = pd.DataFrame(annual_profit.groupby('Tax_number')['annual_profit'].mean())
profit_mean.rename(columns = {'annual_profit': 'firm_avg_annual_profit'}, inplace=True)

# 3. merge profit_mean with df
df = df.merge(profit_mean, how="left", on = 'Tax_number')

# 4. merge annual_profit with df
df = df.merge(annual_profit, how="left", on = ['Tax_number', 'Report Start Date'])

# 5. normalized = annual_profit / firm_avg_annual_profit
df['norm_annual_profit'] = df['annual_profit'] / df['firm_avg_annual_profit']

# check the result
df.head()

,Tax_number,NUTS2,sector,Report Start Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,Regular Operations Income,Extraordinary Income,...,Shareholder's Equity,Long-Term Liabilities,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses,firm_avg_annual_profit,annual_profit,norm_annual_profit
0,10568187.0,Pest,machinery,2003,46109000.0,20296000.0,1531000.0,6000.0,1537000.0,-40000.0,...,6971000.0,0.0,3028000.0,10365000.0,0.327862,14.373938,0.446056,401000.0,1558000.0,3.885287
1,10568187.0,Pest,machinery,2004,40669000.0,17226000.0,6428000.0,34000.0,6462000.0,0.0,...,7145000.0,0.0,2508000.0,9966000.0,0.649643,6.454103,0.367818,401000.0,6462000.0,16.114713
2,10695889.0,Southern Transdanubia,machinery,2003,10530000.0,3330000.0,-949000.0,1000.0,-948000.0,0.0,...,9258000.0,0.0,3955000.0,13329000.0,-0.071623,inf,0.275184,283200.0,-948000.0,-3.347458
3,10695889.0,Southern Transdanubia,machinery,2004,13954000.0,2638000.0,2510000.0,-480000.0,2030000.0,0.0,...,10809000.0,0.0,3942000.0,14811000.0,0.131700,inf,0.214123,283200.0,1946000.0,6.871469
4,10695889.0,Southern Transdanubia,machinery,2005,1027000.0,760000.0,-455000.0,121000.0,-334000.0,-5000000.0,...,7177000.0,0.0,1046000.0,8223000.0,0.569024,inf,0.478891,283200.0,4666000.0,16.475989


In [6]:
# Get annual profit changes
# 1. group by for avg, to avoid duplicated norm_annual_profit
df1 = pd.DataFrame(df.groupby(['Tax_number', 'Report Start Date'])['norm_annual_profit'].mean())

# 2. calculate difference between rows by group
df1['change_annual_profit'] = df1['norm_annual_profit'].diff()

# 3. merge df1 with df
df = df.merge(df1, how="left", on = ['Tax_number', 'Report Start Date'])

# 4. check the result
df.head()

,Tax_number,NUTS2,sector,Report Start Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,Regular Operations Income,Extraordinary Income,...,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses,firm_avg_annual_profit,annual_profit,norm_annual_profit_x,norm_annual_profit_y,change_annual_profit
0,10568187.0,Pest,machinery,2003,46109000.0,20296000.0,1531000.0,6000.0,1537000.0,-40000.0,...,3028000.0,10365000.0,0.327862,14.373938,0.446056,401000.0,1558000.0,3.885287,3.885287,NaN
1,10568187.0,Pest,machinery,2004,40669000.0,17226000.0,6428000.0,34000.0,6462000.0,0.0,...,2508000.0,9966000.0,0.649643,6.454103,0.367818,401000.0,6462000.0,16.114713,16.114713,12.229426
2,10695889.0,Southern Transdanubia,machinery,2003,10530000.0,3330000.0,-949000.0,1000.0,-948000.0,0.0,...,3955000.0,13329000.0,-0.071623,inf,0.275184,283200.0,-948000.0,-3.347458,-3.347458,NaN
3,10695889.0,Southern Transdanubia,machinery,2004,13954000.0,2638000.0,2510000.0,-480000.0,2030000.0,0.0,...,3942000.0,14811000.0,0.131700,inf,0.214123,283200.0,1946000.0,6.871469,6.871469,10.218927
4,10695889.0,Southern Transdanubia,machinery,2005,1027000.0,760000.0,-455000.0,121000.0,-334000.0,-5000000.0,...,1046000.0,8223000.0,0.569024,inf,0.478891,283200.0,4666000.0,16.475989,16.475989,9.604520


In [7]:
# Calculate industrial avg
# 1. group by
industrial_avg = pd.DataFrame(df.groupby(['sector','Report Start Date'])['norm_annual_profit_x'].mean())

# 2. rename column to industrial avg
industrial_avg.rename(columns = {'norm_annual_profit_x': 'industrial_avg'}, inplace=True)

# 3. get industrial avg changes
industrial_avg['change_industrial_avg'] = industrial_avg['industrial_avg'].diff()

# 4. merge industrial_avg with df
df = df.merge(industrial_avg, how="left", on = ['sector','Report Start Date'])
# ? How to do differencing within groups

# 5. check the result
df.head()

,Tax_number,NUTS2,sector,Report Start Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,Regular Operations Income,Extraordinary Income,...,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses,firm_avg_annual_profit,annual_profit,norm_annual_profit_x,norm_annual_profit_y,change_annual_profit,industrial_avg,change_industrial_avg
0,10568187.0,Pest,machinery,2003,46109000.0,20296000.0,1531000.0,6000.0,1537000.0,-40000.0,...,0.327862,14.373938,0.446056,401000.0,1558000.0,3.885287,3.885287,NaN,3.403352,0.232325
1,10568187.0,Pest,machinery,2004,40669000.0,17226000.0,6428000.0,34000.0,6462000.0,0.0,...,0.649643,6.454103,0.367818,401000.0,6462000.0,16.114713,16.114713,12.229426,2.868598,-0.534755
2,10695889.0,Southern Transdanubia,machinery,2003,10530000.0,3330000.0,-949000.0,1000.0,-948000.0,0.0,...,-0.071623,inf,0.275184,283200.0,-948000.0,-3.347458,-3.347458,NaN,3.403352,0.232325
3,10695889.0,Southern Transdanubia,machinery,2004,13954000.0,2638000.0,2510000.0,-480000.0,2030000.0,0.0,...,0.131700,inf,0.214123,283200.0,1946000.0,6.871469,6.871469,10.218927,2.868598,-0.534755
4,10695889.0,Southern Transdanubia,machinery,2005,1027000.0,760000.0,-455000.0,121000.0,-334000.0,-5000000.0,...,0.569024,inf,0.478891,283200.0,4666000.0,16.475989,16.475989,9.604520,3.256890,0.388292


In [8]:
df.isnull().sum().sort_values(ascending=True)
# ? why there are so many Nans in ind avg

Tax_number                                   0
sector                                       0
Report Start Date                            0
annual_profit                                0
firm_avg_annual_profit                       0
Profit or Loss of The Year IS                0
Total Liabilities                            1
Total Assets                                 1
Gross Income                                 7
norm_annual_profit_y                        26
norm_annual_profit_x                        26
Profit After Tax IS                         36
Financial Operations Income                 37
Shareholder's Equity                        88
ROA                                         96
Manufacturing_Costs_to_Total_Expenses      256
Regular Operations Income                  336
NUTS2                                      379
Total Net Sales                            705
Cost of Goods Sold                         871
Inventory_turnover                        1259
Long-Term Lia

In [9]:
# create a list of conditions
conditions = [
    (df['change_industrial_avg'] < 0) & (df['change_annual_profit'] > 0),
    (df['change_industrial_avg'] == 0) & (df['change_annual_profit'] > 0),
    (df['change_industrial_avg'] < 0) & (df['change_annual_profit'] == 0),
    (df['change_industrial_avg'] > 0) & (df['change_annual_profit'] > 0),
    (df['change_industrial_avg'] == 0) & (df['change_annual_profit'] == 0),
    (df['change_industrial_avg'] < 0) & (df['change_annual_profit'] < 0),
    (df['change_industrial_avg'] > 0) & (df['change_annual_profit'] == 0),
    (df['change_industrial_avg'] == 0) & (df['change_annual_profit'] < 0),
    (df['change_industrial_avg'] > 0) & (df['change_annual_profit'] < 0)
    ]
# create a list of the values to assign for each condition
values = [5, 
          4, 4, 
          3, 3, 3, 
          2, 2,
          1]
# create a new column and use np.select to assign values to it using our lists as arguments
df['reactive_resilience'] = np.select(conditions, values)

# 0 represents Nan, fill 0 with nan
df['reactive_resilience'].replace(0, np.NaN)

# check results
df.head()

,Tax_number,NUTS2,sector,Report Start Date,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,Regular Operations Income,Extraordinary Income,...,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses,firm_avg_annual_profit,annual_profit,norm_annual_profit_x,norm_annual_profit_y,change_annual_profit,industrial_avg,change_industrial_avg,reactive_resilience
0,10568187.0,Pest,machinery,2003,46109000.0,20296000.0,1531000.0,6000.0,1537000.0,-40000.0,...,14.373938,0.446056,401000.0,1558000.0,3.885287,3.885287,NaN,3.403352,0.232325,0
1,10568187.0,Pest,machinery,2004,40669000.0,17226000.0,6428000.0,34000.0,6462000.0,0.0,...,6.454103,0.367818,401000.0,6462000.0,16.114713,16.114713,12.229426,2.868598,-0.534755,5
2,10695889.0,Southern Transdanubia,machinery,2003,10530000.0,3330000.0,-949000.0,1000.0,-948000.0,0.0,...,inf,0.275184,283200.0,-948000.0,-3.347458,-3.347458,NaN,3.403352,0.232325,0
3,10695889.0,Southern Transdanubia,machinery,2004,13954000.0,2638000.0,2510000.0,-480000.0,2030000.0,0.0,...,inf,0.214123,283200.0,1946000.0,6.871469,6.871469,10.218927,2.868598,-0.534755,5
4,10695889.0,Southern Transdanubia,machinery,2005,1027000.0,760000.0,-455000.0,121000.0,-334000.0,-5000000.0,...,inf,0.478891,283200.0,4666000.0,16.475989,16.475989,9.604520,3.256890,0.388292,3


In [10]:
# Get a simplified dataframe
finaldf = df[['sector', 'NUTS2', 'Tax_number', 'Report Start Date', 'reactive_resilience', 
               'Total Net Sales', 
               'Cost of Goods Sold', 
               'Gross Income', 
               'Financial Operations Income', 
               'Regular Operations Income', 
               'Extraordinary Income', 
               'Profit After Tax IS', 
               'Profit or Loss of The Year IS', 
               'Total Assets',
               "Shareholder's Equity",
               'Long-Term Liabilities',
               'Current Liabilities',
               'Total Liabilities',
               'ROA',
               'Inventory_turnover',
               'Manufacturing_Costs_to_Total_Expenses']]
finaldf.head()

,sector,NUTS2,Tax_number,Report Start Date,reactive_resilience,Total Net Sales,Cost of Goods Sold,Gross Income,Financial Operations Income,Regular Operations Income,...,Profit After Tax IS,Profit or Loss of The Year IS,Total Assets,Shareholder's Equity,Long-Term Liabilities,Current Liabilities,Total Liabilities,ROA,Inventory_turnover,Manufacturing_Costs_to_Total_Expenses
0,machinery,Pest,10568187.0,2003,0,46109000.0,20296000.0,1531000.0,6000.0,1537000.0,...,1558000.0,1558000.0,4752000.0,6971000.0,0.0,3028000.0,10365000.0,0.327862,14.373938,0.446056
1,machinery,Pest,10568187.0,2004,5,40669000.0,17226000.0,6428000.0,34000.0,6462000.0,...,6462000.0,6462000.0,9947000.0,7145000.0,0.0,2508000.0,9966000.0,0.649643,6.454103,0.367818
2,machinery,Southern Transdanubia,10695889.0,2003,0,10530000.0,3330000.0,-949000.0,1000.0,-948000.0,...,-948000.0,-948000.0,13236000.0,9258000.0,0.0,3955000.0,13329000.0,-0.071623,inf,0.275184
3,machinery,Southern Transdanubia,10695889.0,2004,5,13954000.0,2638000.0,2510000.0,-480000.0,2030000.0,...,1946000.0,1946000.0,14776000.0,10809000.0,0.0,3942000.0,14811000.0,0.131700,inf,0.214123
4,machinery,Southern Transdanubia,10695889.0,2005,3,1027000.0,760000.0,-455000.0,121000.0,-334000.0,...,4666000.0,4666000.0,8200000.0,7177000.0,0.0,1046000.0,8223000.0,0.569024,inf,0.478891


In [11]:
finaldf.to_csv('df_for_modelling.csv')

In [24]:
# Get firms worth investment
# 1. calculate average resilience scores
avg_resilience = pd.DataFrame(finaldf.groupby('Tax_number')['reactive_resilience'].mean())
avg_resilience.rename(columns = {'reactive_resilience': 'avg_resilience'}, inplace=True)

In [25]:
# 2. keep existing firms
# 2.1 get a list of firms that still exist
firm_list = finaldf[finaldf['Report Start Date'] == '2020']
firm_list = firm_list[['sector', 'NUTS2', 'Tax_number']]

# 2.2 exclude firms not on the list by inner joining avg_resilience with firm_list
final_firm_list = pd.merge(firm_list, avg_resilience, how = 'inner', on = 'Tax_number')

# check the result
final_firm_list.head()

,sector,NUTS2,Tax_number,avg_resilience
0,machinery,Budapest,1.002306e+10,2.941176
1,comp_electr,Budapest,1.002379e+10,2.777778
2,basic_metals,Northern Great Plain,1.003010e+10,1.842105
3,comp_electr,Budapest,1.017433e+10,2.823529
4,comp_electr,Pest,1.017433e+10,2.823529


In [26]:
# rank firms within industries
final_firm_list["rank_per_ind"] = final_firm_list.groupby("sector")["avg_resilience"].rank("dense", ascending=False)

# select NO.1 in each industry
top_firms_per_ind = final_firm_list[final_firm_list['rank_per_ind'] == 1]
top_firms_per_ind.drop_duplicates(subset=['Tax_number'], inplace = True)
top_firms_per_ind

C:\Users\Abby\miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,sector,NUTS2,Tax_number,avg_resilience,rank_per_ind
48,machinery,Pest,1.030822e+10,3.500000,1.0
312,machinery,Pest,1.080012e+10,3.500000,1.0
2304,wood_paper_furni,Pest,1.322583e+10,3.692308,1.0
2513,agri,Northern Hungary,1.348550e+10,2.937500,1.0
2520,text_leather,Southern Great Plain,1.349712e+10,3.687500,1.0
2687,comp_electr,Budapest,1.375085e+10,3.357143,1.0
3014,chemicals,Budapest,1.434224e+10,3.692308,1.0
3103,chemicals,Southern Great Plain,1.460317e+10,3.692308,1.0
3546,machinery,Northern Great Plain,2.307105e+10,3.500000,1.0
3599,machinery,Pest,2.316813e+10,3.500000,1.0


In [27]:
# rank firms within regions
final_firm_list["rank_per_reg"] = final_firm_list.groupby("NUTS2")["avg_resilience"].rank("dense", ascending=False)

# select NO.1 in each industry
top_firms_per_reg = final_firm_list[final_firm_list['rank_per_reg'] == 1]
top_firms_per_reg.drop_duplicates(subset=['Tax_number'], inplace = True)
top_firms_per_reg

C:\Users\Abby\miniconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,sector,NUTS2,Tax_number,avg_resilience,rank_per_ind,rank_per_reg
995,wood_paper_furni,Western Transdanubia,1.146948e+10,3.578947,6.0,1.0
1441,wood_paper_furni,Northern Hungary,1.206438e+10,3.666667,2.0,1.0
2233,chemicals,Southern Transdanubia,1.312498e+10,3.600000,4.0,1.0
2304,wood_paper_furni,Pest,1.322583e+10,3.692308,1.0,1.0
2901,chemicals,Central Transdanubia,1.407433e+10,3.642857,2.0,1.0
3014,chemicals,Budapest,1.434224e+10,3.692308,1.0,1.0
3103,chemicals,Southern Great Plain,1.460317e+10,3.692308,1.0,1.0
4671,other,NaN,2.611066e+10,3.250000,5.0,1.0
